In [1]:
!pip install tensorflow

# Tensorflow

In [2]:
import tensorflow as tf
X = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float64)
Y = tf.constant([[2, 3, 4], [5, 6, 7]], dtype=tf.float64)
X

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]])>

In [4]:
print(X + 10)
print(X + Y)
print(X * Y)
print(tf.matmul(X, tf.transpose(Y)))
print(tf.square(X))
print(tf.exp(X))
print(tf.reduce_sum(X))

tf.Tensor(
[[11. 12. 13.]
 [14. 15. 16.]], shape=(2, 3), dtype=float64)
tf.Tensor(
[[ 3.  5.  7.]
 [ 9. 11. 13.]], shape=(2, 3), dtype=float64)
tf.Tensor(
[[ 2.  6. 12.]
 [20. 30. 42.]], shape=(2, 3), dtype=float64)
tf.Tensor(
[[20. 38.]
 [47. 92.]], shape=(2, 2), dtype=float64)
tf.Tensor(
[[ 1.  4.  9.]
 [16. 25. 36.]], shape=(2, 3), dtype=float64)
tf.Tensor(
[[  2.71828183   7.3890561   20.08553692]
 [ 54.59815003 148.4131591  403.42879349]], shape=(2, 3), dtype=float64)
tf.Tensor(21.0, shape=(), dtype=float64)


In [5]:
Z = tf.Variable([[2, 2, 2], [2, 2, 2]], dtype=tf.float64)
Z

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[2., 2., 2.],
       [2., 2., 2.]])>

In [7]:
Z.assign(X)
Z[1, 1].assign(12)
Z.assign_add(Y) # +=
Z.assign_sub(X) # -=
Z

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float64, numpy=
array([[ 2.,  3.,  4.],
       [ 5., 13.,  7.]])>

## GradientTape

In [8]:
def f(x1, x2):
    return 2. * tf.square(x1) + 3. * x1 + 4. * x2 + 5.

x1 = tf.Variable(3.)
x2 = tf.Variable(4.)

with tf.GradientTape() as tape:
    y = f(x1, x2)

dy_dx1, dy_dx2 = tape.gradient(y, [x1, x2])
print(dy_dx1, dy_dx2)

tf.Tensor(15.0, shape=(), dtype=float32) tf.Tensor(4.0, shape=(), dtype=float32)


## Zadanie 1
Zastosuj regułę łańcuchową aby obliczyć $\frac{\partial y}{\partial x_1}$ i $\frac{\partial y}{\partial x_2}$ dla:
1. $y = \sigma(f(x_1, x_2))$, gdzie $f(x_1, x_2)= 2 x_1 + 3 x_2$ i $\sigma(x)=\frac{e^x}{1 + e^x}$,
2. $y = g(f(x_1, x_2))$, gdzie $f(x_1, x_2)= x_1^2 + 2 x_2$ i $g(x)=\sin x$.

Następnie oblicz pochodne funkji za pomocą `GradientTape` i porównaj wyniki.

In [9]:
import tensorflow as tf
import numpy as np
def sigma(x):
    return 1. / (1. + tf.exp(-x))

def f(x1, x2):
    return 2. * x1 + 3. * x2

def F(x1, x2):
    return sigma(f(x1, x2))

def F_prime(x1, x2):
    dy_dx1 = 2. * sigma(f(x1, x2)) * (1 - sigma(f(x1, x2)))
    dy_dx2 = 3. * sigma(f(x1, x2)) * (1 - sigma(f(x1, x2)))
    return float(dy_dx1), float(dy_dx2)

def F_prime_tf(x1, x2):
    with tf.GradientTape() as tape:
        y = F(x1, x2)

    dy_dx1, dy_dx2 = tape.gradient(y, [x1, x2])
    return float(dy_dx1), float(dy_dx2)

x1 = tf.Variable(-2.)
x2 = tf.Variable(1.)

print(F_prime(x1, x2))
print(F_prime_tf(x1, x2))


(0.3932238817214966, 0.5898358821868896)
(0.3932238817214966, 0.5898358225822449)


# B

In [37]:
def g(x):
  result = tf.sin(x)
  return result

def f(x1, x2):
  result = x1**2 + 2.*x2
  return result

def function(x1, x2):
  result = g(f(x1, x2))
  return result

def function_derivative(x1, x2):
  dy_dx1 = tf.cos(f(x1, x2)) * 2. * x1
  dy_dx2 = tf.cos(f(x1, x2)) * + 2
  return float(dy_dx1), float(dy_dx2)

def function_derivative_tf(x1, x2):
  with tf.GradientTape() as tape:
    tape.watch(x1)
    tape.watch(x2)
    y = function(x1, x2)

  dy_dx1, dy_dx2 = tape.gradient(y, [x1, x2])
  return float(dy_dx1), float(dy_dx2)

x1 = tf.Variable(-3.)
x2 = tf.Variable(2.)

print(function_derivative(x1, x2))
print(function_derivative_tf(x1, x2))


(-5.444680690765381, 1.8148936033248901)
(-5.444680690765381, 1.8148936033248901)


## Zadanie 2

1. Zmodyfikuj implementację perceptronu z Ćwiczeń 1. tak, aby stosowała sigmoidalną funkcję aktywacji.
2. Zmodyfikuj poniższą przykładową implementację perceptronu tak aby, stosowała `GradientTape` zamiast wzorów do obliczania gradientów.

In [47]:
import numpy as np
import tensorflow as tf

def sigma(z):
    return 1. / (1. + tf.exp(-z))

class Perceptron:
    def __init__(self, learning_rate=0.01, epochs=100):
        self.learning_rate = learning_rate
        self.epochs = epochs

    def predict(self, X):
        Z = tf.linalg.matvec(X, self.weights) + self.bias
        return sigma(Z)

    def fit(self, X, y):
        self.weights = tf.Variable(np.random.rand(X.shape[1]), dtype=tf.float32)
        self.bias = tf.Variable(np.random.rand(), dtype=tf.float32)

        X = tf.constant(X, dtype=tf.float32)

        for _ in range(self.epochs):
            y_pred = self.predict(X)
            loss = tf.reduce_sum(tf.square(y - y_pred))

            w_gradient = -tf.linalg.matvec(tf.transpose(X), (y - y_pred) * y_pred * (1. - y_pred))
            b_gradient = -tf.reduce_sum((y - y_pred) * y_pred * (1. - y_pred))

            self.weights.assign_sub(self.learning_rate * w_gradient)
            self.bias.assign_sub(self.learning_rate * b_gradient)

    def fit_tf_gradient(self, X, y):
        self.weights = tf.Variable(np.random.rand(X.shape[1]), dtype=tf.float32)
        self.bias = tf.Variable(np.random.rand(), dtype=tf.float32)

        X = tf.constant(X, dtype=tf.float32)
        y = tf.constant(y, dtype=tf.float32)

        for _ in range(self.epochs):
            with tf.GradientTape() as tape:
                tape.watch(self.weights)
                tape.watch(self.bias)

                y_pred = self.predict(X)
                loss = tf.reduce_sum(tf.square(y - y_pred))

            gradients = tape.gradient(loss, [self.weights, self.bias])

            # Apply gradients using gradient descent
            self.weights.assign_sub(self.learning_rate * gradients[0])
            self.bias.assign_sub(self.learning_rate * gradients[1])


In [48]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import scale

X, y = load_breast_cancer(return_X_y=True)
X = scale(X)

X = tf.constant(X, dtype=tf.float32)

perceptron = Perceptron(learning_rate=0.01, epochs=100)
perceptron.fit(X, y)
y_pred = perceptron.predict(X) >= 0.5
print(np.mean(y_pred == y))

perceptron_tf_gradient = Perceptron(learning_rate=0.01, epochs=100)
perceptron_tf_gradient.fit_tf_gradient(X, y)
y_pred_tf_gradient = perceptron_tf_gradient.predict(X) >= 0.5
print(np.mean(y_pred_tf_gradient == y))

0.984182776801406
0.9876977152899824
